# Загрузка Партнеров из файла Влада с 2022

In [1]:
import pandas_gbq
import datetime as dt
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2 import service_account
import gspread
import pandas as pd
import pandas_gbq

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly',
             'https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('m2-main-cd9ed0b4e222.json', SCOPES)
gc = gspread.authorize(credentials)

sh = gc.open('VAS - Партнёры')
wk = sh.worksheet('Ответы на форму (1)')
list_of_dicts = wk.get_all_records()
source = pd.DataFrame(list_of_dicts)

In [ ]:
df = source[['Отметка времени','Название агентства','Регион','Сумма договора','Дата старта','Выберите тип размещение на площадке', 'Какой номер для подмены используем в Mango? (Если не используем написать "Без подменника")', 'Владелец фида в ЕБА (owner_id)']].copy()
df

In [ ]:
df = df.rename(columns = {
 'Отметка времени':'timestamp_mark',
 'Название агентства':'agency', 
 'Регион':'region',
 'Сумма договора': 'deal_amount',
 'Дата старта': 'start_date',
 'Выберите тип размещение на площадке': 'placement_type',
 'Какой номер для подмены используем в Mango? (Если не используем написать "Без подменника")':'phone',
 'Владелец фида в ЕБА (owner_id)':'joo'
}).fillna(0)

In [ ]:
df['start_date'] = pd.to_datetime(df['start_date'], format='%d.%m.%Y')
df['deal_amount'] = df['deal_amount'].astype(int)
df['timestamp_mark'] = pd.to_datetime(df['timestamp_mark'])
df = df[df['placement_type'].isin(['Первичное размещение', 'Пролонгация текущего договора'])]
df = df.reset_index(drop=True)
df = df[df['timestamp_mark']!='']

In [ ]:
df['deal_amount'] = df['deal_amount'] / 30
df['days'] = 30
df

In [ ]:
df['agency'].unique()

In [ ]:
df['agency'].nunique()

In [ ]:
df = df[['agency','region','start_date','deal_amount', 'phone','joo','days']].copy()
df

In [ ]:
df['start_date'] = pd.to_datetime(df['start_date'], format='%d.%m.%Y')

agencies = df['agency'].unique()

for a in agencies:

    single_agency = df.query("agency == @a").sort_values('start_date')

    for index, row in single_agency.iterrows():
        _row = df.loc[[index]] # пересчитываем из оригинального df на случай если уже изменили это значение на предыдущих итерациях цикла
        start = _row['start_date'].iloc[0]
        in_a_deal = start + dt.timedelta(days = 30)
        overlaps = single_agency.query(
            "`start_date` < @in_a_deal and `start_date` > @start")

        if (len(overlaps)):

            for i, ov in overlaps.iterrows():

                print('Setting row {} to {}'.format(i, in_a_deal))
                df.at[i,'start_date'] = in_a_deal

# print(df)

In [ ]:
df['days'] = df['days'].astype(int)
cols = list(df.columns)+['date_spend']
res = []
for i in df.itertuples():    
    for date_diff in range(i.days):
        standart_row = list(i[1:])
        date_spend = i.start_date + dt.timedelta(days=date_diff)
        standart_row.append(date_spend)
        res.append(standart_row)
        
common = pd.DataFrame(res, columns = cols)

In [ ]:
common

In [ ]:
common['deal_amount'] = common['deal_amount'].astype(float)
common = common[common['date_spend'] >= '2022-01-01']
common

In [ ]:
df = common.copy()
df['agency'] = df['agency'].str.lower()
df = df.groupby(['agency','region','phone','joo']).agg(sum=('deal_amount', 'sum')).reset_index()
df.head()

In [ ]:
df = df.sort_values(by=['region','phone'])

In [ ]:
df = df.drop(df.columns[[4]], axis=1).copy()

In [ ]:
df.head()

In [ ]:
df['agency'] = df['agency'].astype(str)
df['region'] = df['region'].astype(str)
df['phone'] = df['phone'].astype(str)
df['joo'] = df['joo'].astype(int)

In [ ]:
t1 =  df.values.tolist()

In [ ]:
sh = gc.open("VAS. Расчет по звонкам")
wk = sh.worksheet('Звонки_update_2023')
wk.update('A8', t1)

# 1. ClickHouse via JOO

In [ ]:
from clickhouse_driver.client import Client
import datetime
import pandas

start = (datetime.datetime.today()-datetime.timedelta(days=7)).date()
end = (datetime.datetime.today()-datetime.timedelta(days=1)).date() 
print(start)
print(end)

In [ ]:
# НЕТ ДАННЫХ В МТС БАЗЕ ЗА ПЕРИОД 5-19 декабря 2022

HOST = '' #удалила креды
PORT = ''
PASS = ''
DB = ''
USER = ''
db_url = 'http://' + USER + ':' + PASS + '@' + HOST + ':' + PORT + '/' + DB
def init_client():
    print('Init db connection on ' + db_url)
    return Client.from_url(db_url)
client = init_client()
res = client.execute(f'''SELECT 
       caller_number as caller,
       external_number as reciever,
       start_time as start_dt,
       offer_owner_id  as joo,
       talk_duration
from doppelganger.mts_stats
where external_number is not Null and start_dt between '{start}' and '{end}' ''')

cols = ['caller','reciever','start_dt','joo', 'talk_dur']
results = pandas.DataFrame(res, columns = cols)

In [ ]:
results.sort_values(by='start_dt').head()
results

In [ ]:
results['start_dt'] = results['start_dt'].astype(str)
results = results.sort_values(by='start_dt').copy()

import re
def clear(text):
    text = re.sub('\+[0-9]+', ' ', text)
    text = re.sub('\.[0-9]+', ' ', text)
    text = re.sub('\ :[0-9]+', ' ', text)
    text = re.sub('\ -[0-9]+', ' ', text)
    text = re.sub('\ :[0-9]+', '', str(text))
    text = re.sub('\:[0-9]+\:[0-9]+', '', text)
    text = re.sub('\ [0-9]+', '', text)
    return text.strip()

results['start_dt'] = [clear(words) for words in results['start_dt']]
results.head(5)

In [ ]:
results['joo'].unique()

In [ ]:
results = results[(results.talk_dur > 30)]
results

In [ ]:
def seven(phone):
    phone = '7' + phone
    return phone.strip()
results['reciever'] = [seven(phone) for phone in results['reciever']]
results['caller'] = [seven(phone) for phone in results['caller']]

In [ ]:
import datetime as dt
import pandas as pd
results['start_dt'] = pd.to_datetime(results['start_dt'], format='%Y-%m-%d')
results['isoweek'] = results['start_dt'].dt.isocalendar().week

In [ ]:
results = results[['isoweek', 'caller', 'reciever', 'joo']]
results.rename(columns={'caller': 'Кто звонил', 'reciever': 'Набранный номер', 'joo': 'Участники',}, inplace=True)
click_house = results.copy()
click_house

# 2. Mango via phone

In [ ]:
# Удалила креды

import csv    

x = r'C:\Users\yakovlevaev\Downloads\Mango-Office_400200144_30_01_2023_13_14.csv'
#path= f'mog/{x}'
with open(x, newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';', quotechar='"')
    doc = list(spamreader)
pd_calls = pd.DataFrame(doc[2:], columns = [i.replace('"','').replace('=','') for i in doc[1]])

In [ ]:
cols = ['Кто звонил', 'Набранный номер', 'Участники']
for i in cols:
    pd_calls[i] = pd_calls[i].apply(lambda x: x.replace('"','').replace('=',''))
pd_calls

In [ ]:
pd_calls['Длительность, сек'] = pd_calls['Длительность, сек'].astype(int)
pd_calls = pd_calls[(pd_calls['Длительность, сек'] > 30)]
pd_calls = pd_calls.loc[~pd_calls['Набранный номер'].str.contains('.*@.*', regex=True, flags=re.I)].reset_index()
pd_calls.head()

In [ ]:
pd_calls = pd_calls.drop(columns = ['Длительность, сек' , 'Стоимость, руб', 'Стоимость без услуг, руб'])
pd_calls['Время'] = pd_calls['Время'].apply(lambda x: x[:10])
pd_calls['Время'] =pd_calls['Время'].apply(lambda x: datetime.datetime.strptime(x,'%d.%m.%Y').date())
pd_calls = pd_calls.sort_values(by=['Время'], ignore_index =True)

In [ ]:
pd_calls.drop('Тип вызова', axis=1, inplace=True)

In [ ]:
pd_calls['Время'] = pd.to_datetime(pd_calls['Время'], format='%Y-%m-%d')
pd_calls['isoweek'] = pd_calls['Время'].dt.isocalendar().week

In [ ]:
pd_calls = pd_calls[['isoweek', 'Кто звонил', 'Набранный номер', 'Участники']]

In [ ]:
mango = pd_calls.copy()
mango

In [ ]:
# на всякий случай проверка на дубли --- в Манго табле остаются только те номера, которых нет в КликХаусе
click_house['Набранный номер'] = click_house['Набранный номер'].astype(str)
mango['Набранный номер'] = mango['Набранный номер'].astype(str)
mango = mango[~mango['Набранный номер'].isin(click_house['Набранный номер'])].reset_index(drop=True)
mango

In [ ]:
clh = click_house[['isoweek', 'Участники']].copy()
clh.rename(columns={'Участники': 'joo_or_phone'}, inplace=True)
mng = mango[['isoweek', 'Набранный номер']].copy()
mng.rename(columns={'Набранный номер': 'joo_or_phone'}, inplace=True)

In [ ]:
# склеиваем
frames = [clh, mng]
common = pd.concat(frames)
common['calls'] = 1
common

# 3. Расхлопы via JOO

In [ ]:
from datetime import date
today = date.today()
week = today.isocalendar()[1] - 1
week

In [ ]:
from clickhouse_key import ch_key
from clickhouse_driver import Client

client = Client(ch_key['db_host'],
               user=ch_key['db_user'],
               password=ch_key['db_password'],
               verify=False,
               database=ch_key['db_name'], settings={'use_numpy': True})

In [ ]:
re, columns = client.execute(f'''
select
    toISOWeek(dateHourMinute) as isoweek,
    EXTRACT(eventAction, '.*_(.*)_.*') AS joo_or_phone,
    CAST(FLOOR(sum(uniqueEvents) * 0.11) AS INT) as calls
from ga.RAW_EVENTS
where match(eventlabel, 'UPUNICLOP_.*_(card|serp)')
    AND match(eventCategory, 'second_sell_true_unknown.*')
    and length(EXTRACT(eventAction, '.*_(.*)_.*')) > 0
    AND toISOYear(dateHourMinute) >= 2023
    AND toISOWeek(dateHourMinute) >= {week}
group by isoweek, joo_or_phone
order by isoweek, joo_or_phone
''', with_column_types = True)

In [ ]:
col = ['isoweek', 'joo_or_phone', 'calls']
res = pd.DataFrame(re,columns=col)
res.head(4)

In [ ]:
res.info()

In [ ]:
clops = res.copy()

In [ ]:
clops['joo_or_phone'] = clops['joo_or_phone'].astype(str)
clh['joo_or_phone'] = clh['joo_or_phone'].astype(str)

In [ ]:
# обязательная проверка на дубли
clops = clops[~clops['joo_or_phone'].isin(clh['joo_or_phone'])].reset_index(drop=True)
clp = clops[(clops[['joo_or_phone']] != '').any(axis=1)].reset_index(drop=True)
clp

In [ ]:
frames = [common, clp]
total = pd.concat(frames).groupby(['isoweek','joo_or_phone']).agg(calls=('calls', 'sum')).reset_index()
total

In [ ]:
# a1_vas = [total.columns.values.tolist()] + total.values.tolist()
a1_vas = total.values.tolist()

In [ ]:
sh = gc.open("VAS. Расчет по звонкам")
wk = sh.worksheet('Расхлопы_2023')
#wk.update('G1', a1_vas)
wk.append_rows(a1_vas, table_range="G1")

# Часть 11% от расхлопов

In [ ]:
result, columns = client.execute('''
select
    toISOWeek(dateHourMinute) as isoweek,
    EXTRACT(eventAction, '(.*?)_.*') as eventaction,
    sum(totalEvents) as rashlop
from ga.RAW_EVENTS
where eventlabel IN ('ClCardSellSecondVasPhoneClickAll', 'ClSerpSellSecondVasPhoneClick')
    and dateHourMinute >= '2023-01-02' and length(EXTRACT(eventAction, '(.*?)_.*')) > 0
group by isoweek, eventaction
order by isoweek, eventaction
''', with_column_types = True)

col = ['isoweek', 'eventaction', 'rashlop']
r = pd.DataFrame(result, columns=col)
r.head(4)

In [ ]:
a1_vas = [r.columns.values.tolist()] + r.values.tolist()

In [ ]:
sh = gc.open("VAS. Расчет по звонкам")
wk = sh.worksheet('Расхлопы_2023')
wk.update('A1', a1_vas)